In [2]:
import numpy as np
import pandas as pd
from scipy.stats import linregress
from sklearn.cross_decomposition import CCA
import scipy.stats as stats
from itertools import combinations

# 소수를 e로 표현하지 않도록 하기
pd.options.display.float_format = "{:.5f}".format

In [3]:
lib_2007 = pd.read_csv("final_data/lib_2007.csv")
lib_2008 = pd.read_csv("final_data/lib_2008.csv")
lib_2009 = pd.read_csv("final_data/lib_2009.csv")
lib_2010 = pd.read_csv("final_data/lib_2010.csv")
lib_2011 = pd.read_csv("final_data/lib_2011.csv")
lib_2012 = pd.read_csv("final_data/lib_2012.csv")
lib_2013 = pd.read_csv("final_data/lib_2013.csv")
lib_2014 = pd.read_csv("final_data/lib_2014.csv")
lib_2015 = pd.read_csv("final_data/lib_2015.csv")
lib_2016 = pd.read_csv("final_data/lib_2016.csv")
lib_2017 = pd.read_csv("final_data/lib_2017.csv")
lib_2018 = pd.read_csv("final_data/lib_2018.csv")
lib_2019 = pd.read_csv("final_data/lib_2019.csv")
lib_2020 = pd.read_csv("final_data/lib_2020.csv")
lib_2021 = pd.read_csv("final_data/lib_2021.csv")

In [4]:
train_lib = pd.concat([lib_2007,lib_2008,lib_2009,lib_2010,lib_2011,lib_2012,lib_2013,lib_2014,lib_2015])
valid_lib = pd.concat([lib_2016,lib_2017,lib_2018])
test_lib = pd.concat([lib_2019,lib_2020,lib_2021])

In [5]:
# train, test set의 데이터 수 구하기
print(f"train set의 데이터 수 : {len(train_lib)}")
print(f"valid set의 데이터 수 : {len(valid_lib)}")
print(f"test set의 데이터 수 : {len(test_lib)}")

train set의 데이터 수 : 6808
valid set의 데이터 수 : 2848
test set의 데이터 수 : 3483


In [6]:
# 사분위수를 활용하여 비율 충족도의 이상치 행 제거하기
def del_outlier(df, col):
    q1 = df[col].quantile(0.25)
    q3 = df[col].quantile(0.75)
    iqr = q3 - q1 
    boundary = 1.5 * iqr 

    upper_index = df[df[col] > q3 + boundary].index
    lower_index = df[df[col] < q1 - boundary].index 

    df.drop(upper_index, inplace = True)
    df.drop(lower_index, inplace = True)

    return df

del_outlier(train_lib, "Future_fullfillment")
del_outlier(valid_lib, "Future_fullfillment")
del_outlier(test_lib, "Future_fullfillment") 

# 이상치 제거 후 train, test set의 데이터 수 구하기
print(f"이상치 제거 후 train set의 데이터 수 : {len(train_lib)}")
print(f"이상치 제거 후 valid set의 데이터 수 : {len(valid_lib)}")
print(f"이상치 제거 후 test set의 데이터 수 : {len(test_lib)}")

이상치 제거 후 train set의 데이터 수 : 4014
이상치 제거 후 valid set의 데이터 수 : 2398
이상치 제거 후 test set의 데이터 수 : 3004


In [7]:
# train set과 test set 내에 있는 변수들에 대해 로그 변환하기

for col in train_lib.columns[:-1]:
    train_lib[col] = np.log1p(train_lib[col])
    
for col in valid_lib.columns[:-1]:
    valid_lib[col] = np.log1p(valid_lib[col])

for col in test_lib.columns[:-1]:
    test_lib[col] = np.log1p(test_lib[col])

In [8]:
# X_train, Y_train 구성하기
X_train = train_lib.loc[:, (train_lib.columns != "Future_fullfillment")]
Y_train = train_lib["Future_fullfillment"]

# X_valid, Y_valid 구성하기
X_valid = valid_lib.loc[:, (valid_lib.columns != "Future_fullfillment")]
Y_valid = valid_lib["Future_fullfillment"]

# X_test, Y_test 구성하기
X_test = test_lib.loc[:, (test_lib.columns != "Future_fullfillment")]
Y_test = test_lib["Future_fullfillment"]

In [9]:
# 비율충족도가 0일 경우 해당 값을 0.000001로 변환하기
Y_train[Y_train == 0] = 0.000001
Y_valid[Y_valid == 0] = 0.000001
Y_test[Y_test == 0] = 0.000001

In [10]:
result_df = pd.DataFrame(columns=['Feature1', 'Feature2', 'Pearson_Corr', 'Pearson_p-value', 'Spearman_Corr', 'Spearman_p-value', 'Kendall_Corr', 'Kendall_p-value'])

In [11]:
features = ['dom_books','for_books','local_mat','nonbook_mat','elec_mat','serials','ann_growth','ann_weeding', 'lib_site_area', 'lib_build_area', 'total_seats', 'user_comps', 'self_srv_machines', 'full_time', 'part_time', 'support_staff', 'total_budget', 'acq_budget', 'open_days', 'avg_week_hours', 'reg_members', 'lib_visitors', 'service_recip', 'borrowers', 'loans', 'interlib_loans', 'info_serv_requests', 'web_access', 'user_ed_sessions', 'user_ed_partic', 'prog_sessions', 'prog_partic', 'disab_mat', 'vuln_group_budget', 'total_settlement', 'acq_budget_settlement'
]

In [13]:
# 특성 조합 생성
feature_pairs = list(combinations(features, 2))

In [18]:
# CCA 및 상관관계 분석
result_list = []
for X1, X2 in feature_pairs:
    X1_data = X_test[[X1]]
    X2_data = X_test[[X2]]
    
    cca = CCA(scale=False, n_components=1)
    cca.fit(X1_data, X2_data)
    X1_c, X2_c = cca.transform(X1_data, X2_data)
    
    corr_coefficient, p_value = np.corrcoef(X1_c[:, 0], X2_c[:, 0])[0, 1], linregress(X1_c[:, 0], X2_c[:, 0]).pvalue
    
    rho, p_val = stats.pearsonr(X1_c[:, 0], X2_c[:, 0])
    rho_spearman, p_val_spearman = stats.spearmanr(X1_c[:, 0], X2_c[:, 0])
    rho_kendall, p_val_kendall = stats.kendalltau(X1_c[:, 0], X2_c[:, 0])
    
    result_list.append([X1, X2, rho, p_val, rho_spearman, p_val_spearman, rho_kendall, p_val_kendall])

result_df = pd.DataFrame(result_list, columns=['Feature1', 'Feature2', 'Pearson_Corr', 'Pearson_p-value', 'Spearman_Corr', 'Spearman_p-value', 'Kendall_Corr', 'Kendall_p-value'])



In [19]:
# CSV 파일 저장
result_df.to_csv('cca_results/test_cca.csv', index=False)